Lecture 13 will cover further go into examples on how to tie the GUI-widgets to the application logic by looking into more on the signal handling in Qt.

First a bit of standard stuff needed to work with Qt

In [1]:
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import sys
qt_app = QApplication(sys.argv)

# Signals

Lets start by creating a simpel model that emits a signal when the data has been modified.

In [2]:
class MyCounterModel(QObject):
    # We create a new signal
    new_count = pyqtSignal()
    warning = pyqtSignal(str,)
    
    def __init__(self):
        super().__init__()
        self.count = 1

    def value(self):
        return self.count

    def increment(self, val=1):
        if val > 100:
            self.warning.emit('Input value to high!')
        else:
            self.count += val
            self.new_count.emit()


In [3]:
class MyCounterView(QWidget):
    def __init__(self, model):
        super().__init__()
        self.button = QPushButton("Add 1")
        self.doubler = QPushButton("Double")
        self.label = QLabel()

        layout = QHBoxLayout()
        layout.addWidget(self.button)
        layout.addWidget(self.doubler)
        layout.addWidget(self.label)
        self.setLayout(layout)

        # Connect logic:
        # View part:
        self.model = model
        model.new_count.connect(self.update_value)
        model.warning.connect(self.display_warning)
        
        # Controller part:
        def add_one():
            model.increment()
        self.button.clicked.connect(add_one)
        
        def double_up():
            x = model.value()
            model.increment(x)       
        self.doubler.clicked.connect(double_up)
        
        self.update_value()  # Make sure it's up-to-date from the start.
    
    def update_value(self):
        self.label.setText("Value is " + str(self.model.value()))
        
    def display_warning(self, text):
        print(text)  # Display this to the user, somehow

In [4]:
qt_app = QApplication.instance()

counter = MyCounterModel()
view = MyCounterView(counter)
view.show()

qt_app.exec_()

Input value to high!


0

Lets start adressing the strange parts:
* The signal is declared as a static variable (class variable).
* The pyqtSignal() object doesn't actually have a connect or emit method.

It's all "magic" that the initialization of QObject takes care of. You don't need know how it happens, just how to use it.
*In order to use pyqtSignal()'s the class needs to inherit from QObject.*

# Creating reuseable Model and View components

Many games render the whole view using a single drawing area (not unlike the CardView class illustrated last lecture), and in these cases, there is really only 1 view that displays one 1 model, though for a more complex software it becomes completely unfeasable to keep track of all the state.

As such, it is typically better to create smaller Model's and View's that have a clear and simple purpose, and simply rely on those in the rest of the code.

Such components is typically simpler, with a clear purpose, and can be reuseable.

## Revisiting the AddOne game from lecture 12
We can revisit the previous game, but tackle the signals and model design with smaller models and views that take care of themselves:

In [5]:
class PointModel(QObject):
    # This point model has 1 purpose: ensuring the signal is emitted whenever the value is changed
    new_value = pyqtSignal()

    def __init__(self):
        super().__init__()
        self.value = 0
    
    def add_one(self):
        self.value += 1
        self.new_value.emit()

    def clear(self):
        self.value = 0
        self.new_value.emit()


class Player:
    def __init__(self, name):
        self.name = name
        # The Player class does not need to concern itself with any signals
        # since the PointModel handles that itself!
        self.total = PointModel()


class AddOneGame(QObject):
    winner = pyqtSignal((str,))
    
    def __init__(self):
        super().__init__()
        self.players = [Player(name) for name in ['Micke', 'Kim', 'Thomas']]
    
    def player_click(self, index):
        # We must make sure not to directly modify total.value, else we won't let it emit its signal!
        # But the code in the "complex" AddOneGame is now much simpler;
        # No need to keep track of updating signals for all the values here, just pure game logic!
        self.players[index].total.add_one()
        self.check_winner()
    
    def reset(self):
        for player in self.players:
            player.total.clear()
    
    def check_winner(self):
        for player in self.players:
            if player.total.value >= 10:
                self.winner.emit(player.name + " won!")
                self.reset()
                break

**Emphasis**:
Note how the models above only contain state and logic. No graphical elements at all, no information on how it is displayed. It might just as well be played on the command line!

Below we will now introduce the View(s), which contain **no logic at all**. They make no assumptions on when the points are updated. They only decide how to display the state of the model to the user, and present the options for how to control the model.

In [6]:
class PointView(QLabel):
    # The point view only needs to know about the PointModel and how to best display it, listening to the signal
    def __init__(self, point_model: PointModel):
        super().__init__()
        self.point_model = point_model
        self.point_model.new_value.connect(self.update_label)
        self.update_label()  # Initial refresh of textfield

    def update_label(self):
        self.setText(f'Points: {self.point_model.value}')


class PlayerView(QGroupBox):
    def __init__(self, player: Player, game: AddOneGame, player_number: int):
        super().__init__(player.name)

        # Connect button event to register in the game model:
        button = QPushButton('Add one')
        def click_event():
            return game.player_click(player_number)
        button.clicked.connect(click_event)

        # Set the layout of this player view:
        vbox = QVBoxLayout()
        vbox.addWidget(PointView(player.total))
        vbox.addWidget(button)
        self.setLayout(vbox)
    

class GameView(QWidget):
    def __init__(self, game_model):
        super().__init__()
        
        vbox = QVBoxLayout()
        for i, player in enumerate(game_model.players):
            vbox.addWidget(PlayerView(player, game, i))

        reset_button = QPushButton('Reset')
        reset_button.clicked.connect(game_model.reset)
        
        vbox.addWidget(reset_button)
        
        self.setLayout(vbox)
        self.show()

        # Connect the last signal:
        self.game = game_model
        game_model.winner.connect(self.alert_winner)

    def alert_winner(self, text: str):
        msg = QMessageBox()
        msg.setText(text)
        msg.exec()

In [7]:
qt_app = QApplication.instance()
game = AddOneGame()
view = GameView(game)
qt_app.exec_()

0

# When to split off things to its own "sub-model"

This is a more subjective design question. Certainly, noone would split a complex number into a model for the real and imaginary parts respectively.

For example, strongly associated values

- Coordinate; longitud and latitude
- Brush shape in a drawing application; radius, sharpness
- Player money; how much money available and how much currently betted
- List of recent filenames

it seems logical that these could be their own class.

But what about When we have a collection of losely connected things, such as a "Player" model that contains the money and the cards on hand?
These often don't change at the same time, and have basically no overlap in functionality.
It is almost certainly neater, cleaner, and easier to reason about the code if these were kept separately.

Keeping separate signals also allows us to avoid unnecessary updating of variables that might not have changed; making for a snappier application

## Another game example

In [8]:
# This is a simple game. The players take turn add 1 through 3, until they reach 20. The player who gets 20 (or higher) wins.
# This simple game doesn't motivate the complex design here, but the purpose was to illustrate the local states.
class GameState(QObject):
    # We might have multiple signals! One for updates
    data_changed = pyqtSignal()
    # and one for messaging
    game_message = pyqtSignal((str,)) # (x,) is the notation for a tuple with just one value!

    def __init__(self, players):
        super().__init__()
        self.running = False
        self.players = players
        self.player_turn = -1
        self.total = 0
        
    def start(self):
        if self.running:
            self.game_message.emit("Can't start game. Game already running")
        
        self.running = True
        self.player_turn = 0
        self.total = 0
        self.players[self.player_turn].set_active(True)
        self.data_changed.emit()

    def add(self, num):
        # Called when a player adds a value (this also switches the players turn)
        self.total += num
        if self.total >= 20:
            winner = self.players[self.player_turn]
            self.game_message.emit("Player {} won!".format(winner.name))
            winner.won()
            self.total = 0

        self.players[self.player_turn].set_active(False)
        self.player_turn = (self.player_turn + 1) % len(self.players)
        self.players[self.player_turn].set_active(True)
        self.data_changed.emit()

# A simple player state. It keeps track of the score.
class PlayerState(QObject):

    data_changed = pyqtSignal()
    
    def __init__(self, name):
        super().__init__()
        self.name = name
        self.wins = 0
        self.active = False
    
    def set_active(self, active):
        self.active = active
        self.data_changed.emit()

    def won(self):
        self.wins += 1
        self.data_changed.emit()

In [9]:
class PlayerView(QGroupBox):
    def __init__(self, player, game):
        super().__init__(player.name)
        self.player = player
        
        layout = QVBoxLayout()
        self.setLayout(layout)

        self.wins = QLabel()
        layout.addWidget(self.wins)

        self.buttons = []
        for b in range(3):
            button = QPushButton("Add {}".format(b+1))
            self.buttons.append(button)
            layout.addWidget(button)
            #button.clicked.connect( lambda : game.add(b+1) )

        def add_1(): game.add(1)
        def add_2(): game.add(2)
        def add_3(): game.add(3)
        self.buttons[0].clicked.connect(add_1)
        self.buttons[1].clicked.connect(add_2)
        self.buttons[2].clicked.connect(add_3)

        player.data_changed.connect(self.update)
        self.update()

    def update(self):
        self.wins.setText("Wins: " + str(self.player.wins))
        for b in self.buttons:
            b.setEnabled(self.player.active)

In [10]:
class GameView(QWidget):
    def __init__(self, game):
        super().__init__()
    
        self.game = game
        
        layoutv = QVBoxLayout()
        self.setLayout(layoutv)

        self.total_label = QLabel("uninitialized")
        layoutv.addWidget(self.total_label)
        
        layouth = QHBoxLayout()
        layoutv.addLayout(layouth)
        
        self.player_views = []
        for p in game.players:
            player_view = PlayerView(p, game)
            self.player_views.append(player_view)
            layouth.addWidget(player_view)

        game.game_message.connect(self.alert_user)
        game.data_changed.connect(self.update)
        game.start() # We start as soon as we get a view!
        self.update()

    def alert_user(self, text):
        # A method like this is nice to have for showing if the game is over, 
        # or warn about faulty input.
        box = QMessageBox()
        box.setText(text)
        box.exec_()
        
    def update(self):
        self.total_label.setText("Total: " + str(self.game.total))

In [12]:
qt_app = QApplication.instance()

model = GameState([PlayerState("Micke"), PlayerState("Thomas")])
view = GameView(model)
view.show()

qt_app.exec_()

0

The view and controller (as they are typically bundled) has 3 means of communication;
* View: A reference to the model of which it can ask what to display (e.g. what cards the player has, how much money is in the pot, whose turn it is)
* View: A signal to connect the "update" method to (which may or may not send information)
* Controller: Methods on the Model to pass information back after user interaction.

As we can see, even if we have a widget that represents one player-model, it doens't mean that widget reports changes back to that player-model. *
Typical in games (and many other applications), we have a overreaching game logic which needs to take the input.

## Tell or ask?

What is best?
* The Model sends a empty signal and the View(s) ask the Model for the information it needs.
* The Model sends the information in the signal

Probably no right answer, but there are some scenarions when passing a temporary state along might be the only reasonable option;
* Message/warning to the user
* The change of a value (e.g. how much it increased)
* Which index in a list model that was modified
